# Import required packages

In [2]:
#pip install opencv-python

In [3]:
# update-notebook-instance --recolourize --

In [4]:
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import gc

In [5]:
import tensorflow as tf

In [6]:
from keras.preprocessing.image import ImageDataGenerator

# reading files from s3 bucket

In [7]:
training_data_uri = 's3://recolorising/small_set_LAB_3000.npy'

In [8]:
#Check the contents of the uri
# !aws s3 ls $training_data_uri/

In [9]:
from s3fs.core import S3FileSystem
s3 = S3FileSystem()

In [10]:
key = 'small_set_LAB_3000.npy'
bucket = 'recolorising'

full_img = np.load(s3.open('{}/{}'.format(bucket, key)))

In [11]:
#full_img.shape

(3000, 224, 224, 3)

converting lab into rgb for effective downsampling

In [ ]:
full_img_rgb = []
for i in range(0, full_img.shape[0]):
    if i == 0:
        full_img_rgb = cv2.cvtColor(np.array(full_img[i,:,:,:]),cv2.COLOR_Lab2RGB)
        #print(full_img_rgb.shape)
        full_img_rgb = np.expand_dims(full_img_rgb,0)
    else:
        full_img_rgb = np.append(full_img_rgb,
                    np.expand_dims(cv2.cvtColor(np.array(full_img[i,:,:,:]),cv2.COLOR_Lab2RGB),0),axis=0)
full_img_rgb = tf.convert_to_tensor(full_img_rgb)

Resizing the images to 128*128 p

In [ ]:
#img = full_img_rgb[2, :, :, :]
#img = cv2.cvtColor(np.array(img), cv2.COLOR_Lab2RGB)
#plt.imshow(img)

In [ ]:
full_img = tf.image.resize_with_pad(
                full_img_rgb,
                128,
                128,
                "bilinear"
                )

freeing up memory

In [ ]:
del full_img_rgb
gc.collect()

img = full_img[2,:,:,:]
#img = cv2.cvtColor(np.array(img), cv2.COLOR_Lab2RGB)
plt.imshow(np.array(img).astype('int'))

COnverting RGB back into Lab

In [ ]:
full_img_lab = []
for i in range(0,full_img.shape[0]):
     if i==0:
          full_img_lab = cv2.cvtColor(np.array(full_img[i,:,:,:]),cv2.COLOR_RGB2Lab)
          #print(full_img_lab.shape)
          full_img_lab=np.expand_dims(full_img_lab,0)
     else:
          full_img_lab = np.append(full_img_lab,
                                         np.expand_dims(cv2.cvtColor(np.array(full_img[i,:,:,:]),cv2.COLOR_RGB2Lab),0),axis=0)
full_img = tf.convert_to_tensor(full_img_lab)

In [ ]:
del full_img_lab, ab, l
gc.collect()

# train and test split 70:30

In [ ]:
train = full_img[:2200,:,:,:]
test = full_img[2200:,:,:,:]

Normalising the picture data

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255)
datagen = ImageDataGenerator(featurewise_center=True,
                            featurewise_std_normalization=True)
datagen.fit(test)

In [ ]:
train_iterator = datagen.flow(train, batch_size=64)
test_iterator = datagen.flow(test, batch_size=64)
print('Batches train=%d, test=%d' % (len(train_iterator),
                                     len(test_iterator)))

In [ ]:
gray_train = train[:,:,:,1]
gray_test = test[:,:,:,1]

Write data to folder

In [ ]:
np.save('Preprocessed/train.npy', train)
np.save('Preprocessed/test.npy', test)
np.save('Preprocessed/gray_train.npy', gray_train)
np.save('Preprocessed/gray_test.npy', gray_test)

Write data to s3 bucket

In [ ]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = "recolorising"
prefix = "processed/"

role = sagemaker.get_execution_role()

In [ ]:
inputs = sagemaker_session.upload_data(path="pytorch-data", bucket=bucket, key_prefix=prefix)
print("input spec (in this case, just an S3 path): {}".format(inputs))